In [1]:
import keras
from keras import layers
import numpy as np

latent_dim = 32
height = 32
width = 32
channels = 3

generator_input = keras.Input(shape=(latent_dim))

x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16,16,128))(x)

x = layers.Conv2D(256,5,padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2DTranspose(256,4,strides=2,padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256,5,padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256,5,padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(channels,7,activation='tanh',padding='same')(x)
generator = keras.models.Model(generator_input,x)
generator.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
dense (Dense)                (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32768)             0         
_________________________________________________________________
reshape (Reshape)            (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 32, 32, 256)       104883

In [2]:
discriminator_input = layers.Input(shape=(height,width,channels))
x = layers.Conv2D(128,3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128,4,strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128,4,strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128,4,strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

x = layers.Dropout(0.4)(x)

x = layers.Dense(1,activation='sigmoid')(x)

discriminator = keras.models.Model(discriminator_input,x)
discriminator.summary()

discriminator_optimizer = keras.optimizers.RMSprop(
    lr=0.0008,
    clipvalue=1.0,   #梯度裁剪范围
    decay=1e-8
)

discriminator.compile(optimizer=discriminator_optimizer,
                     loss='binary_crossentropy')


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 6, 6, 128)         0   

In [3]:
discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input,gan_output)
gan_optimizer = keras.optimizers.RMSprop(lr=0.0004,clipvalue=1.0,decay=1e-8)
gan.compile(optimizer=gan_optimizer,loss='binary_crossentropy')

gan.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
model (Model)                (None, 32, 32, 3)         6264579   
_________________________________________________________________
model_1 (Model)              (None, 1)                 790913    
Total params: 7,055,492
Trainable params: 6,264,579
Non-trainable params: 790,913
_________________________________________________________________


In [8]:
import os
from keras.preprocessing import image

(x_train,y_train),(_,_) = keras.datasets.cifar10.load_data()
x_train = x_train[y_train.flatten()==6]  #选取青蛙类的图片
x_train = x_train.reshape(
    (x_train.shape[0],)+(height,width,channels)).astype('float32') / 255.

iterations = 10000
batch_size = 20
save_dir = 'C:/Users/admin/Desktop/save'


start = 0
for step in range(iterations):
    random_latent_vectors = np.random.normal(size=(batch_size,latent_dim))
    
    generated_images = generator.predict(random_latent_vectors)
    
    stop = start +batch_size
    real_images = x_train[start:stop]
    combined_images  = np.concatenate([generated_images,real_images])
    
    labels = np.concatenate([np.ones((batch_size,1)),np.zeros((batch_size,1))])#合并标签，区分真实和虚假的图像
    labels += 0.05 * np.random.random(labels.shape)   #向标签中添加随机噪声
    
    random_latent_vectors = np.random.normal(size=(batch_size,latent_dim))
    
    d_loss = discriminator.train_on_batch(combined_images,labels)#潜在空间随机采样
    
    misleading_targets = np.zeros((batch_size,1))#合并标签，全部为“真实图像”（撒谎）
    
    a_loss = gan.train_on_batch(random_latent_vectors,misleading_targets) #通过gan模型来训练生成器（此时判别器权重冻结）
    
    start += batch_size
    
    if start>len(x_train) - batch_size:
        start = 0 
    
    if start%100 == 0:
        gan.save_weights('ganh5')
        
        print('discriminator loss:',d_loss)
        print('adversarial loss:',a_loss)
        
        img = image.array_to_img(generated_images[0] * 255.,scale=False)
        img.save(os.path.join(save_dir,'real_frog'+str(step) + '.png'))

discriminator loss: 0.6804517507553101
adversarial loss: 0.8001454472541809
discriminator loss: 0.6884753704071045
adversarial loss: 0.8724398612976074
discriminator loss: 0.6959547996520996
adversarial loss: 0.6565869450569153
discriminator loss: 0.6845732927322388
adversarial loss: 0.6459585428237915
discriminator loss: 0.6611843109130859
adversarial loss: 0.6948649883270264
discriminator loss: 0.6768186688423157
adversarial loss: 0.7527177929878235
discriminator loss: 0.6814996004104614
adversarial loss: 0.9363007545471191
discriminator loss: 0.6554340124130249
adversarial loss: 1.3918521404266357
discriminator loss: 0.612889289855957
adversarial loss: 10.43153190612793
discriminator loss: 0.6072022914886475
adversarial loss: 1.219878077507019
discriminator loss: 0.60586017370224
adversarial loss: 1.0576335191726685
discriminator loss: 0.5165767669677734
adversarial loss: 1.2193138599395752
discriminator loss: 0.6311007738113403
adversarial loss: 0.9802371859550476
discriminator los

discriminator loss: 0.6950937509536743
adversarial loss: 0.7451688647270203
discriminator loss: 0.6904798150062561
adversarial loss: 0.7342816591262817
discriminator loss: 0.6890357732772827
adversarial loss: 0.7550393342971802
discriminator loss: 0.6979223489761353
adversarial loss: 0.7418153882026672
discriminator loss: 0.6928324699401855
adversarial loss: 0.6339680552482605
discriminator loss: 0.5937002897262573
adversarial loss: 1.8025712966918945
discriminator loss: 0.6334632635116577
adversarial loss: 2.177726984024048
discriminator loss: 0.6766344904899597
adversarial loss: 0.753694474697113
discriminator loss: 0.6364372968673706
adversarial loss: 1.8014549016952515
discriminator loss: 0.7151225209236145
adversarial loss: 0.7198340892791748
discriminator loss: 0.725996196269989
adversarial loss: 0.7013722658157349
discriminator loss: 0.6613941192626953
adversarial loss: 0.8548614382743835
discriminator loss: 0.6957372426986694
adversarial loss: 0.7699633836746216
discriminator l

discriminator loss: 0.7109553813934326
adversarial loss: 0.7514515519142151
discriminator loss: 0.6973440051078796
adversarial loss: 0.7510355710983276
discriminator loss: 0.6870300769805908
adversarial loss: 0.7358801960945129
discriminator loss: 0.6955901980400085
adversarial loss: 0.7649493217468262
discriminator loss: 0.6909741163253784
adversarial loss: 0.7810004949569702
discriminator loss: 0.6880954504013062
adversarial loss: 0.7804490923881531
discriminator loss: 0.6997806429862976
adversarial loss: 0.7094114422798157
discriminator loss: 0.6993629932403564
adversarial loss: 0.7583024501800537
discriminator loss: 0.6969841122627258
adversarial loss: 0.7490944266319275
discriminator loss: 0.6942736506462097
adversarial loss: 0.7981063723564148
discriminator loss: 0.6892529726028442
adversarial loss: 0.7569422721862793
discriminator loss: 0.69404536485672
adversarial loss: 0.752998948097229
discriminator loss: 0.7007941007614136
adversarial loss: 0.7309977412223816
discriminator l

discriminator loss: 0.6911714673042297
adversarial loss: 0.7535741329193115
discriminator loss: 0.696108341217041
adversarial loss: 0.7375346422195435
discriminator loss: 0.7069867849349976
adversarial loss: 0.7651578187942505
discriminator loss: 0.6883427500724792
adversarial loss: 0.7771375775337219
discriminator loss: 0.6776517033576965
adversarial loss: 0.7568303346633911
discriminator loss: 0.7112415432929993
adversarial loss: 0.7365485429763794
discriminator loss: 0.6851309537887573
adversarial loss: 0.7260561585426331
discriminator loss: 0.6937015056610107
adversarial loss: 0.7746405601501465
discriminator loss: 0.6848088502883911
adversarial loss: 0.7839385271072388
discriminator loss: 0.7002246379852295
adversarial loss: 0.7584958076477051
discriminator loss: 0.6820908784866333
adversarial loss: 0.6664832234382629
discriminator loss: 0.7289971113204956
adversarial loss: 0.8087860941886902
discriminator loss: 0.6819700002670288
adversarial loss: 0.7603439688682556
discriminator

discriminator loss: 0.6819315552711487
adversarial loss: 0.7838290333747864
discriminator loss: 0.708335280418396
adversarial loss: 0.7795625329017639
discriminator loss: 0.6901372075080872
adversarial loss: 0.759290874004364
discriminator loss: 0.6938593983650208
adversarial loss: 0.7698208689689636
discriminator loss: 0.6932989954948425
adversarial loss: 0.782868504524231
discriminator loss: 0.7041352391242981
adversarial loss: 0.7420684099197388
discriminator loss: 0.6956170797348022
adversarial loss: 0.7277313470840454
discriminator loss: 0.705265998840332
adversarial loss: 0.7301298379898071
discriminator loss: 0.713497519493103
adversarial loss: 0.7455806732177734
discriminator loss: 0.6961414217948914
adversarial loss: 0.7450776696205139
discriminator loss: 0.6865781545639038
adversarial loss: 0.735380232334137
discriminator loss: 0.6936119794845581
adversarial loss: 0.7314649820327759
discriminator loss: 0.6950471997261047
adversarial loss: 0.7562882304191589
discriminator loss

...

discriminator loss: 0.6921101808547974
adversarial loss: 0.8426021337509155
discriminator loss: 0.6963154077529907
adversarial loss: 0.8434651494026184
discriminator loss: 0.7112584114074707
adversarial loss: 0.8688094019889832
discriminator loss: 0.6975957155227661
adversarial loss: 0.7720930576324463
discriminator loss: 0.6994616389274597
adversarial loss: 0.755206823348999
discriminator loss: 0.6773868799209595
adversarial loss: 0.6967993974685669
discriminator loss: 0.7275270223617554
adversarial loss: 0.8445823788642883
discriminator loss: 0.6985887289047241
adversarial loss: 0.7654271125793457
discriminator loss: 0.6905990839004517
adversarial loss: 0.7577582597732544
discriminator loss: 0.6901004314422607
adversarial loss: 0.7499827146530151
discriminator loss: 0.6927923560142517
adversarial loss: 0.7361456751823425
discriminator loss: 0.6939229965209961
adversarial loss: 0.7954612970352173
discriminator loss: 0.6935204267501831
adversarial loss: 0.7494286298751831
discriminator

discriminator loss: 0.6792080998420715
adversarial loss: 0.7513790726661682
discriminator loss: 0.6835254430770874
adversarial loss: 0.7014811635017395
discriminator loss: 0.701212465763092
adversarial loss: 0.7442212104797363
discriminator loss: 0.6943564414978027
adversarial loss: 0.7506996393203735
discriminator loss: 0.698531985282898
adversarial loss: 0.7831305265426636
discriminator loss: 0.6802288293838501
adversarial loss: 0.7383569478988647
discriminator loss: 0.6961270570755005
adversarial loss: 0.9142246246337891
discriminator loss: 0.6906430125236511
adversarial loss: 0.8380632400512695
discriminator loss: 0.6803848147392273
adversarial loss: 0.6914147138595581
discriminator loss: 0.7022041082382202
adversarial loss: 0.7607074975967407
discriminator loss: 0.6871312260627747
adversarial loss: 0.7248174548149109
discriminator loss: 0.7196177244186401
adversarial loss: 0.9706274271011353
discriminator loss: 0.6902733445167542
adversarial loss: 0.7444955110549927
discriminator 

discriminator loss: 0.6783279180526733
adversarial loss: 0.7850536108016968
discriminator loss: 0.7203507423400879
adversarial loss: 0.7546122670173645
discriminator loss: 0.6578537821769714
adversarial loss: 0.7824351191520691
discriminator loss: 0.6677248477935791
adversarial loss: 0.70225989818573
discriminator loss: 0.7162253260612488
adversarial loss: 0.7381574511528015
discriminator loss: 0.7347121238708496
adversarial loss: 0.8371647000312805
discriminator loss: 0.7421621084213257
adversarial loss: 0.8711307644844055
discriminator loss: 0.6927083134651184
adversarial loss: 0.7631500959396362
discriminator loss: 0.7118264436721802
adversarial loss: 0.6085221171379089
discriminator loss: 0.6710848808288574
adversarial loss: 0.7990988492965698
discriminator loss: 0.7070937156677246
adversarial loss: 0.7732560634613037
discriminator loss: 0.6673740148544312
adversarial loss: 0.7647389769554138
discriminator loss: 0.6975900530815125
adversarial loss: 0.7941392660140991
discriminator 

discriminator loss: 0.6506956219673157
adversarial loss: 0.7836408615112305
discriminator loss: 0.6977554559707642
adversarial loss: 0.7347389459609985
discriminator loss: 0.6871179938316345
adversarial loss: 0.8118554353713989
discriminator loss: 0.6765917539596558
adversarial loss: 0.811204731464386
discriminator loss: 0.7371050119400024
adversarial loss: 0.9307071566581726
discriminator loss: 0.6724643111228943
adversarial loss: 0.8011951446533203
discriminator loss: 0.810002326965332
adversarial loss: 1.0102283954620361
discriminator loss: 0.672991156578064
adversarial loss: 0.7364500761032104
discriminator loss: 0.7070558667182922
adversarial loss: 0.8022632598876953
discriminator loss: 0.7089777588844299
adversarial loss: 0.9169901013374329
discriminator loss: 0.6810142397880554
adversarial loss: 0.9890039563179016
discriminator loss: 0.7068511247634888
adversarial loss: 0.5771182775497437
discriminator loss: 0.6802743673324585
adversarial loss: 0.7150118947029114
discriminator l

discriminator loss: 0.8382786512374878
adversarial loss: 1.065398931503296
discriminator loss: 0.8378673791885376
adversarial loss: 1.3141207695007324
discriminator loss: 0.736138105392456
adversarial loss: 0.8825219869613647
discriminator loss: 0.683112621307373
adversarial loss: 0.7765393257141113
discriminator loss: 0.7152776718139648
adversarial loss: 0.75301593542099
discriminator loss: 0.6904168128967285
adversarial loss: 0.7918487787246704
discriminator loss: 0.6469890475273132
adversarial loss: 0.6862929463386536
discriminator loss: 0.7712273597717285
adversarial loss: 0.9079526662826538
discriminator loss: 0.7324085235595703
adversarial loss: 0.7301236391067505
discriminator loss: 0.7058058977127075
adversarial loss: 0.8214131593704224
discriminator loss: 0.6837207078933716
adversarial loss: 0.7663372159004211
discriminator loss: 0.6163470149040222
adversarial loss: 0.9016455411911011
discriminator loss: 0.6790189743041992
adversarial loss: 0.7853097915649414
discriminator los